# Recipe 3: Creating a Custom Table

This is an advanced recipe that should be followed only after mastering the basic recipe.
This recipe shows how to prepare a custom table.

In [ ]:
# Install conda and taxcalc if in Google Colab.
import sys
if 'google.colab' in sys.modules and 'taxcalc' not in sys.modules:
    !wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
    !bash Miniconda3-latest-Linux-x86_64.sh -bfp /usr/local
    # Append path to be able to run packages installed with conda
    # This must correspond to the conda Python version, which may differ from
    # the base Colab Python installation.
    sys.path.append('/usr/local/lib/python3.7/site-packages')
    # Install PSL packages from Anaconda
    !yes | conda install -c PSLmodels taxcalc

In [1]:
import taxcalc as tc
import numpy as np

# use publicly-available CPS input file
recs = tc.Records.cps_constructor()

# specify Calculator object for static analysis of current-law policy
pol = tc.Policy()
calc = tc.Calculator(policy=pol, records=recs)

CYR = 2020

# calculate aggregate current-law income tax liabilities for cyr
calc.advance_to_year(CYR)
calc.calc_all()

# tabulate custom table showing number of filing units receiving EITC
# and the average positive EITC amount by IRS-SOI AGI categories
vardf = calc.dataframe(['s006', 'c00100', 'eitc'])
vardf = tc.add_income_table_row_variable(vardf, 'c00100', tc.SOI_AGI_BINS)
gbydf = vardf.groupby('table_row', as_index=False)

Filing Units Receiving EITC and Average Positive EITC by AGI Category

In [2]:
# print AGI table with ALL row at bottom
results = '{:23s}\t{:8.3f}\t{:8.3f}'
colhead = '{:23s}\t{:>8s}\t{:>8s}'
print(colhead.format('AGI Category', 'Num(#M)', 'Avg($K)'))
tot_recips = 0.
tot_amount = 0.
idx = 0
for grp_interval, grp in gbydf:
    recips = grp[grp['eitc'] > 0]['s006'].sum() * 1e-6
    tot_recips += recips
    amount = (grp['eitc'] * grp['s006']).sum() * 1e-9
    tot_amount += amount
    if recips > 0:
        avg = amount / recips
    else:
        avg = np.nan
    glabel = '[{:.8g}, {:.8g})'.format(grp_interval.left, grp_interval.right)
    print(results.format(glabel, recips, avg))
    idx += 1
avg = tot_amount / tot_recips
print(results.format('ALL', tot_recips, avg))

AGI Category           	 Num(#M)	 Avg($K)
[-9e+99, 1)            	   0.053	   1.358
[1, 5000)              	   1.144	   0.515
[5000, 10000)          	   2.004	   1.387
[10000, 15000)         	   3.140	   1.869
[15000, 20000)         	   2.640	   3.345
[20000, 25000)         	   2.259	   4.046
[25000, 30000)         	   2.021	   3.775
[30000, 40000)         	   3.813	   2.491
[40000, 50000)         	   2.140	   1.299
[50000, 75000)         	   0.425	   0.570
[75000, 100000)        	   0.000	     nan
[100000, 200000)       	   0.000	     nan
[200000, 500000)       	   0.000	     nan
[500000, 1000000)      	   0.000	     nan
[1000000, 1500000)     	   0.000	     nan
[1500000, 2000000)     	   0.000	     nan
[2000000, 5000000)     	   0.000	     nan
[5000000, 10000000)    	   0.000	     nan
[10000000, 9e+99)      	   0.000	     nan
ALL                    	  19.638	   2.415
